In [1]:
import numpy as np
import cv2
from glob import glob
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from pathlib import Path
import re
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from tqdm import tqdm
from utils import l2_norm
import logging

tf.get_logger().setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# Load Model

In [2]:
### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'CosHead' # ''ArcHead', CosHead', 'SphereHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception
#     But if you are trying to use NasNet, please check this issue first: https://github.com/keras-team/keras-applications/issues/78
#         We manually download the weight file and explicitly load it in models.py file
# Backbones w/o pretrained weights:
#     MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
backbone_type = 'ResNet50' 
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print("train_size: ",train_size)
steps_per_epoch = train_size // batch_size
print("steps_per_epoch: ",steps_per_epoch)
val_size = dataset_len - train_size
print("val_size: ",val_size)
validation_steps = val_size // batch_size
print("validation_steps: ",validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False
projection_head = False  # True
dgx = True

version = "Check"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
    
# collect loss in checkpoints
# file_list = []
# for files in glob(base_dir+save_name):
#     if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
#         loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
#     file_list.append( loss  )
# file_list.sort()

# load_file_name = []
# for files in glob(base_dir+save_name):
#     if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
#         pass
#     elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
#         load_file_name = files
# best_checkpoint = load_file_name.split('.data')[0]
# print("best checkpoint: ",best_checkpoint)

train_size:  4658122
steps_per_epoch:  36391
val_size:  1164531
validation_steps:  9097


In [3]:
best_checkpoint = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CosHead_check/Check/e_20_l_8.25856876373291.ckpt"
print("best checkpoint: ",best_checkpoint)
weight_file = best_checkpoint
model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False,
                         projection_head=projection_head)
model.load_weights(weight_file)
model.summary()

best checkpoint:  /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CosHead_check/Check/e_20_l_8.25856876373291.ckpt
Model: "arcface_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 112, 112, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
OutputLayer (Functional)     (None, 512)               16787968  
Total params: 40,375,680
Trainable params: 40,312,320
Non-trainable params: 63,360
_________________________________________________________________


# For IJBC

In [4]:
import os
import pickle

import matplotlib
import pandas as pd

matplotlib.use('Agg')
import matplotlib.pyplot as plt
import timeit
import sklearn
import argparse
from sklearn.metrics import roc_curve, auc

from pathlib import Path
import sys
import warnings

import cv2
import numpy as np
from skimage import transform as trans

In [5]:
meta_path = "/raid/workspace/jbpark/IJB-C/IJBC/meta/"

In [6]:
def read_template_media_list(path):
    # ijb_meta = np.loadtxt(path, dtype=str)
    ijb_meta = pd.read_csv(path, sep=' ', header=None).values
    templates = ijb_meta[:, 1].astype(np.int)
    medias = ijb_meta[:, 2].astype(np.int)
    return templates, medias

In [7]:
def read_template_pair_list(path):
    # pairs = np.loadtxt(path, dtype=str)
    pairs = pd.read_csv(path, sep=' ', header=None).values
    # print(pairs.shape)
    # print(pairs[:, 0].astype(np.int))
    t1 = pairs[:, 0].astype(np.int)
    t2 = pairs[:, 1].astype(np.int)
    label = pairs[:, 2].astype(np.int)
    return t1, t2, label

# STEP1:  Load Meta Data

In [8]:
# =============================================================
# load image and template relationships for template feature embedding
# tid --> template id,  mid --> media id
# format:
#           image_name tid mid
# =============================================================
start = timeit.default_timer()
templates, medias = read_template_media_list(meta_path +"ijbc_face_tid_mid.txt")
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Time: 0.28 s. 


In [9]:
templates

array([     1,      1,      1, ..., 187955, 187955, 187955])

In [10]:
medias

array([ 69544,   3720,      3, ..., 111105, 111105, 111105])

In [11]:
def template_encoding(templates, medias, img_norm_feats):
# def template_encoding(templates, medias, img_norm_feats):
    # ==========================================================
    # 1. face image --> l2 normalization.
    # 2. compute media encoding.
    # 3. compute template encoding.
    # 4. save template features.
    # ==========================================================
    print('==> compute template-level feature encoding.')

    uq_temp = np.unique(templates)
    num_temp = len(uq_temp)
    tmp_feats = np.empty((num_temp, 512))

    for c, uqt in enumerate(uq_temp):
        (ind_t,) = np.where(templates == uqt)
#         face_norm_feats = img_norm_feats[ind_t]
        faces_media = medias[ind_t]
        uqm, counts = np.unique(faces_media, return_counts=True)
        media_norm_feats = []

        for u,ct in zip(uqm, counts):
            (ind_m,) = np.where(faces_media == u)
            if ct < 2:
                media_norm_feats += [face_norm_feats[ind_m]]
            else:
                media_norm_feats += [np.sum(face_norm_feats[ind_m], 0, keepdims=True)]

        media_norm_feats = np.array(media_norm_feats)
        media_norm_feats = media_norm_feats / np.sqrt(np.sum(media_norm_feats ** 2, -1, keepdims=True))
        template_norm_feats = np.sum(media_norm_feats, 0)
        tmp_feats[c] = template_norm_feats
        if c % 500 == 0:
            print('-> finish encoding {}/{} templates.'.format(c, num_temp))
    return tmp_feats


In [12]:
# =============================================================
# load template pairs for template-to-template verification
# tid : template id,  label : 1/0
# format:
#           tid_1 tid_2 label
# =============================================================
start = timeit.default_timer()
p1, p2, label = read_template_pair_list(meta_path +"ijbc_template_pair_label.txt")
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Time: 2.64 s. 


In [13]:
p1

array([     1,      1,      1, ..., 171707, 171707, 171707])

In [14]:
p2

array([ 11065,  11066,  11067, ..., 183140, 182146, 185794])

In [15]:
label

array([1, 1, 1, ..., 0, 0, 0])

# STEP2: Get Image Features

In [16]:
def get_embed(rimg, landmark, input_size):
    image_size = (input_size,input_size)

    img = cv2.resize(rimg, dsize=(image_size[1], image_size[0]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_flip = np.fliplr(img)
#     img = np.transpose(img, (2, 0, 1))  # 3*112*112, RGB
#     img_flip = np.transpose(img_flip, (2, 0, 1))
    input_blob = np.zeros((2, image_size[1], image_size[0], 3), dtype=np.uint8)
    input_blob[0] = ((img/255)-0.5)/0.5
    input_blob[1] = ((img_flip/255)-0.5)/0.5
    return input_blob

In [34]:
files[0]
name_lmk_score = files[0].strip().split(' ')
img_name = os.path.join(img_path, name_lmk_score[0])
img = cv2.imread(img_name)

In [39]:
img

array([[[179, 196, 215],
        [181, 198, 217],
        [182, 196, 218],
        ...,
        [187, 194, 219],
        [186, 193, 220],
        [184, 190, 219]],

       [[184, 199, 218],
        [182, 197, 216],
        [181, 195, 217],
        ...,
        [185, 193, 216],
        [185, 192, 219],
        [187, 194, 221]],

       [[182, 196, 215],
        [187, 201, 220],
        [183, 197, 216],
        ...,
        [184, 192, 215],
        [185, 192, 217],
        [192, 199, 224]],

       ...,

       [[158, 164, 183],
        [109, 115, 134],
        [ 77,  81, 100],
        ...,
        [ 97,  89, 100],
        [117, 110, 123],
        [ 98,  94, 106]],

       [[117, 123, 142],
        [ 77,  83, 102],
        [ 70,  71,  91],
        ...,
        [110, 100, 112],
        [121, 112, 125],
        [ 94,  87, 100]],

       [[ 83,  90, 107],
        [ 65,  72,  89],
        [ 67,  68,  88],
        ...,
        [114, 104, 116],
        [116, 107, 120],
        [ 89,  82,  95]]

In [41]:
(img/255)-0.5

array([[[ 0.20196078,  0.26862745,  0.34313725],
        [ 0.20980392,  0.27647059,  0.35098039],
        [ 0.21372549,  0.26862745,  0.35490196],
        ...,
        [ 0.23333333,  0.26078431,  0.35882353],
        [ 0.22941176,  0.25686275,  0.3627451 ],
        [ 0.22156863,  0.24509804,  0.35882353]],

       [[ 0.22156863,  0.28039216,  0.35490196],
        [ 0.21372549,  0.27254902,  0.34705882],
        [ 0.20980392,  0.26470588,  0.35098039],
        ...,
        [ 0.2254902 ,  0.25686275,  0.34705882],
        [ 0.2254902 ,  0.25294118,  0.35882353],
        [ 0.23333333,  0.26078431,  0.36666667]],

       [[ 0.21372549,  0.26862745,  0.34313725],
        [ 0.23333333,  0.28823529,  0.3627451 ],
        [ 0.21764706,  0.27254902,  0.34705882],
        ...,
        [ 0.22156863,  0.25294118,  0.34313725],
        [ 0.2254902 ,  0.25294118,  0.35098039],
        [ 0.25294118,  0.28039216,  0.37843137]],

       ...,

       [[ 0.11960784,  0.14313725,  0.21764706],
        [-0

In [42]:
((img/255)-0.5)/0.5

array([[[ 0.40392157,  0.5372549 ,  0.68627451],
        [ 0.41960784,  0.55294118,  0.70196078],
        [ 0.42745098,  0.5372549 ,  0.70980392],
        ...,
        [ 0.46666667,  0.52156863,  0.71764706],
        [ 0.45882353,  0.51372549,  0.7254902 ],
        [ 0.44313725,  0.49019608,  0.71764706]],

       [[ 0.44313725,  0.56078431,  0.70980392],
        [ 0.42745098,  0.54509804,  0.69411765],
        [ 0.41960784,  0.52941176,  0.70196078],
        ...,
        [ 0.45098039,  0.51372549,  0.69411765],
        [ 0.45098039,  0.50588235,  0.71764706],
        [ 0.46666667,  0.52156863,  0.73333333]],

       [[ 0.42745098,  0.5372549 ,  0.68627451],
        [ 0.46666667,  0.57647059,  0.7254902 ],
        [ 0.43529412,  0.54509804,  0.69411765],
        ...,
        [ 0.44313725,  0.50588235,  0.68627451],
        [ 0.45098039,  0.50588235,  0.70196078],
        [ 0.50588235,  0.56078431,  0.75686275]],

       ...,

       [[ 0.23921569,  0.28627451,  0.43529412],
        [-0

In [17]:
def get_image_feature(img_path, files_list, model):
    batch_size = 2048

    files = files_list
    print('files:', len(files))
    rare_size = len(files) % batch_size
    faceness_scores = []
    batch = 0
    img_feats = np.empty((len(files), 1024), dtype=np.float32)

    batch_data = np.empty((2 * batch_size, input_size, input_size, 3))
    for img_index, each_line in enumerate(files[:len(files) - rare_size]):
        name_lmk_score = each_line.strip().split(' ')
        img_name = os.path.join(img_path, name_lmk_score[0])
        img = cv2.imread(img_name)
        lmk = np.array([float(x) for x in name_lmk_score[1:-1]],
                       dtype=np.float32)
        lmk = lmk.reshape((5, 2))
        input_blob = get_embed(img, lmk, input_size)

        batch_data[2 * (img_index - batch * batch_size)][:] = input_blob[0]
        batch_data[2 * (img_index - batch * batch_size) + 1][:] = input_blob[1]
        if (img_index + 1) % batch_size == 0:
            print('batch', batch)
            img_feats[batch * batch_size:batch * batch_size +
                      batch_size][:] = model.predict(batch_data).reshape([batch_size,2*512])
            batch += 1
        faceness_scores.append(name_lmk_score[-1])

    batch_data = np.empty((2 * rare_size, input_size, input_size, 3))
    
    # batch size로 안나눠지는 남는 것들 처리
    for img_index, each_line in enumerate(files[len(files) - rare_size:]):
        name_lmk_score = each_line.strip().split(' ')
        img_name = os.path.join(img_path, name_lmk_score[0])
        img = cv2.imread(img_name)
        lmk = np.array([float(x) for x in name_lmk_score[1:-1]],
                       dtype=np.float32)
        lmk = lmk.reshape((5, 2))
        input_blob = get_embed(img, lmk, input_size)
        batch_data[2 * img_index][:] = input_blob[0]
        batch_data[2 * img_index + 1][:] = input_blob[1]
        if (img_index + 1) % rare_size == 0:
            print('batch', batch)
            img_feats[len(files) - rare_size:][:] = model.predict(batch_data).reshape([rare_size,2*512])
            batch += 1
        faceness_scores.append(name_lmk_score[-1])
    faceness_scores = np.array(faceness_scores).astype(np.float32)
    # img_feats = np.ones( (len(files), 1024), dtype=np.float32) * 0.01
    # faceness_scores = np.ones( (len(files), ), dtype=np.float32 )
    return img_feats, faceness_scores


In [18]:
# =============================================================
# load image features
# format:
#           img_feats: [image_num x feats_dim] (227630, 512)
# =============================================================
start = timeit.default_timer()
image_path = "/raid/workspace/jbpark/IJB-C"
img_path = '%s/veri_crops' % image_path
print(img_path)
img_list_path = meta_path +"ijbc_name_5pts_score.txt"
img_list = open(img_list_path)
files = img_list.readlines()
# files_list = divideIntoNstrand(files, rank_size)
files_list = files

/raid/workspace/jbpark/IJB-C/veri_crops


In [19]:
# img_feats
# for i in range(rank_size):
img_feats, faceness_scores = get_image_feature(img_path, files_list, model)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))
print('Feature Shape: ({} , {}) .'.format(img_feats.shape[0],
                                          img_feats.shape[1]))


files: 469375
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82
batch 83
batch 84
batch 85
batch 86
batch 87
batch 88
batch 89
batch 90
batch 91
batch 92
batch 93
batch 94
batch 95
batch 96
batch 97
batch 98
batch 99
batch 100
batch 101
batch 102
batch 103
batch 104
batch 105
batch 106
batch 107
batch 108
batch 

# STEP3: Get Template Feature 

In [20]:
def image2template_feature(img_feats=None, templates=None, medias=None):
    # ==========================================================
    # 1. face image feature l2 normalization. img_feats:[number_image x feats_dim]
    # 2. compute media feature.
    # 3. compute template feature.
    # ==========================================================
    unique_templates = np.unique(templates)
    template_feats = np.zeros((len(unique_templates), img_feats.shape[1]))
    for count_template, uqt in enumerate(unique_templates):

        (ind_t,) = np.where(templates == uqt)
        face_norm_feats = img_feats[ind_t]
        face_medias = medias[ind_t]
        unique_medias, unique_media_counts = np.unique(face_medias,
                                                       return_counts=True)
        media_norm_feats = []
        for u, ct in zip(unique_medias, unique_media_counts):
            (ind_m,) = np.where(face_medias == u)
            if ct == 1:
                media_norm_feats += [face_norm_feats[ind_m]]
            else:  # image features from the same video will be aggregated into one feature
                media_norm_feats += [
                    np.mean(face_norm_feats[ind_m], axis=0, keepdims=True)
                ]
        media_norm_feats = np.array(media_norm_feats)
        # media_norm_feats = media_norm_feats / np.sqrt(np.sum(media_norm_feats ** 2, -1, keepdims=True))
        template_feats[count_template] = np.sum(media_norm_feats, axis=0)
        if count_template % 2000 == 0:
            print('Finish Calculating {} template features.'.format(
                count_template))
    # template_norm_feats = template_feats / np.sqrt(np.sum(template_feats ** 2, -1, keepdims=True))
    template_norm_feats = sklearn.preprocessing.normalize(template_feats)
    # print(template_norm_feats.shape)
    return template_norm_feats, unique_templates

In [21]:
# =============================================================
# compute template features from image features.
# =============================================================
start = timeit.default_timer()
# ==========================================================
# Norm feature before aggregation into template feature?
# Feature norm from embedding network and faceness score are able to decrease weights for noise samples (not face).
# ==========================================================
# 1. FaceScore （Feature Norm）
# 2. FaceScore （Detector）
use_flip_test = False
use_norm_score = True
use_detector_score = False

# if use_flip_test:
#     # concat --- F1
#     # img_input_feats = img_feats
#     # add --- F2
#     img_input_feats = img_feats[:, 0:img_feats.shape[1] //
#                                      2] + img_feats[:, img_feats.shape[1] // 2:]
# else:
#     img_input_feats = img_feats[:, 0:img_feats.shape[1] // 2]
img_input_feats = img_feats
    
    
if use_norm_score:
    img_input_feats = img_input_feats
else:
    # normalise features to remove norm information
    img_input_feats = img_input_feats / np.sqrt(
        np.sum(img_input_feats ** 2, -1, keepdims=True))

if use_detector_score:
    print(img_input_feats.shape, faceness_scores.shape)
    img_input_feats = img_input_feats * faceness_scores[:, np.newaxis]
else:
    img_input_feats = img_input_feats

In [22]:
template_norm_feats, unique_templates = image2template_feature(
    img_input_feats, templates, medias)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Finish Calculating 0 template features.
Finish Calculating 2000 template features.
Finish Calculating 4000 template features.
Finish Calculating 6000 template features.
Finish Calculating 8000 template features.
Finish Calculating 10000 template features.
Finish Calculating 12000 template features.
Finish Calculating 14000 template features.
Finish Calculating 16000 template features.
Finish Calculating 18000 template features.
Finish Calculating 20000 template features.
Finish Calculating 22000 template features.
Time: 8.16 s. 


In [23]:
template_norm_feats.shape

(23124, 1024)

In [24]:
unique_templates.shape

(23124,)

# STEP4: Get Template Similarity Scores

In [25]:
def verification(template_norm_feats=None,
                 unique_templates=None,
                 p1=None,
                 p2=None):
    # ==========================================================
    #         Compute set-to-set Similarity Score.
    # ==========================================================
    template2id = np.zeros((max(unique_templates) + 1, 1), dtype=int)
    for count_template, uqt in enumerate(unique_templates):
        template2id[uqt] = count_template

    score = np.zeros((len(p1),))  # save cosine distance between pairs

    total_pairs = np.array(range(len(p1)))
    batchsize = 100000  # small batchsize instead of all pairs in one batch due to the memory limiation
    sublists = [
        total_pairs[i:i + batchsize] for i in range(0, len(p1), batchsize)
    ]
    total_sublists = len(sublists)
    for c, s in enumerate(sublists):
        feat1 = template_norm_feats[template2id[p1[s]]]
        feat2 = template_norm_feats[template2id[p2[s]]]
        similarity_score = np.sum(feat1 * feat2, -1)
        score[s] = similarity_score.flatten()
        if c % 10 == 0:
            print('Finish {}/{} pairs.'.format(c, total_sublists))
    return score

In [26]:
# =============================================================
# compute verification scores between template pairs.
# =============================================================
start = timeit.default_timer()
score = verification(template_norm_feats, unique_templates, p1, p2)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

# In[ ]:
save_path = "result/"
# save_path = result_dir + '/%s_result' % target

if not os.path.exists(save_path):
    os.makedirs(save_path)
target = f'{head_type}_{backbone_type}'
score_save_file = os.path.join(save_path, "%s.npy" % target.lower())
np.save(score_save_file, score)

Finish 0/157 pairs.
Finish 10/157 pairs.
Finish 20/157 pairs.
Finish 30/157 pairs.
Finish 40/157 pairs.
Finish 50/157 pairs.
Finish 60/157 pairs.
Finish 70/157 pairs.
Finish 80/157 pairs.
Finish 90/157 pairs.
Finish 100/157 pairs.
Finish 110/157 pairs.
Finish 120/157 pairs.
Finish 130/157 pairs.
Finish 140/157 pairs.
Finish 150/157 pairs.
Time: 269.46 s. 


In [27]:
score.shape

(15658489,)

In [28]:
score

array([0.99921104, 0.99941251, 0.99952832, ..., 0.99769731, 0.99625762,
       0.95939094])

# STEP5: Compute ROC

In [29]:
def compute_ROC(labels, scores, roc_path):
    print('==> compute ROC.')
    import sklearn.metrics as skm
    from scipy import interpolate
    fpr, tpr, thresholds = skm.roc_curve(labels, scores)
    roc_auc = skm.auc(fpr, tpr)
    print("AUROC: ",roc_auc)
    fpr_levels = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    f_interp = interpolate.interp1d(fpr, tpr)
    tpr_at_fpr = [f_interp(x) for x in fpr_levels]
    roc_txt = f'ROC_{head_type}_{backbone_type}.txt'.lower()
    file = open(f'{save_path}{roc_txt}', 'w')
    for (far, tar) in zip(fpr_levels, tpr_at_fpr):
        print('TAR @ FAR = {} : {}'.format(far, tar))
        file.write('TAR @ FAR = {}: {}\n'.format(far, tar))
    file.close()

In [30]:
compute_ROC(label, score, save_path)

==> compute ROC.
AUROC:  0.5595851197879201
TAR @ FAR = 1e-05 : 1.8819823802409166e-05
TAR @ FAR = 0.0001 : 0.00018819823802409164
TAR @ FAR = 0.001 : 0.0018819823802409164
TAR @ FAR = 0.01 : 0.018487935665332772
TAR @ FAR = 0.1 : 0.139643094544153
